# part 11. apply vs map
이 장은 전처리의 마지막 카드를 다룬다. <br>
대부분의 전처리는 연산(map, 문자열 메서드, 조건 필터링)으로 끝나야 한다.<br>
그게 안 될 때만 apply를 쓴다.

## 1) map의 용도 — 값 → 값 치환
map은 "각 값을 다른 값으로 바꾼다." 한 문장으로 정의된다.
- 입력 값 1개 → 출력 값 1개
- 조건 분기 없음
- 미리 정해진 대응표(딕셔너리)를 사용

<br>

그래서 map은:
- 범주값 통일
- 코드 → 의미
- 의미 → bool

같은 치환 작업에 최적이다.

### 코드로 살펴보자
``` python
Series.map(매핑규칙)

```
이 Series의 각 값을 이 규칙에 따라 다른 값으로 바꿔라

In [1]:
import pandas as pd

df = pd.DataFrame({
    "channel": ["kiosk", "app", "kiosk", "web"]
})

channel_map = {
    "kiosk": "키오스크",
    "app": "앱",
    "web": "웹"
}
print("매핑 전")
print(df)

# channel 컬럼의 값들을 channel_map 기준으로 그대로 치환한다
df["channel_name"] = df["channel"].map(channel_map)
print("\n  매핑 후")
print(df)

매핑 전
  channel
0   kiosk
1     app
2   kiosk
3     web

  매핑 후
  channel channel_name
0   kiosk         키오스크
1     app            앱
2   kiosk         키오스크
3     web            웹


### 자주하는 실수포인트
- 매핑표에 없는 값은 NaN이 된다
- 그래서 항상 누락을 한 번 확인하는 게 좋다

In [2]:
missing = df[df["channel_name"].isna()]
print(missing)

Empty DataFrame
Columns: [channel, channel_name]
Index: []


## 2) apply의 용도 — 규칙 기반 판단
apply는 성격이 완전히 다르다. <br>
한 줄의 규칙으로 안 되는 로직을 적용한다

특징:
- if / else 같은 조건 분기 가능
- 여러 컬럼을 동시에 참고 가능
- 결과는 자유도가 높다

그래서 apply는:
- 복합 조건 판단
- 점수 등급 계산
- 상태 판정

같은 규칙 기반 처리에 사용한다.

### 코드로 살펴보자
```python
# 값 하나씩 처리
Series.apply(함수)

# 행 단위 처리 (여러 컬럼 참조)
DataFrame.apply(함수, axis=1)
```

In [3]:
import pandas as pd

df = pd.DataFrame({"sales": [0, 3000, 12000]})

def grade_sales(x):
    if x == 0:
        return "C"
    elif x < 10000:
        return "B"
    else:
        return "A"

df["grade"] = df["sales"].apply(grade_sales)
print(df)

   sales grade
0      0     C
1   3000     B
2  12000     A


In [4]:
import pandas as pd

df = pd.DataFrame([
    {"sales": 12000, "paid": True},
    {"sales": 5000,  "paid": True},
    {"sales": 8000,  "paid": False},
])


def final_label(row):
    if row["paid"] is False:
        return "FAIL"
    if row["sales"] >= 10000:
        return "A"
    return "B"


print("행 단위")
df["label"] = df.apply(final_label, axis=1)
print(df)

행 단위
   sales   paid label
0  12000   True     A
1   5000   True     B
2   8000  False  FAIL


### apply 사용 시 주의사항
apply는 강력하지만 주의해서 사용해야 한다.
- 처리 속도가 느리다
- 코드가 길어지고 복잡해지기 쉽다
- 벡터화된 Pandas 연산의 장점을 잃는다

그래서 apply는 가장 먼저 쓰는 도구가 아니라, 마지막에 선택하는 도구다.

## 3) 선택 기준 요약
apply와 map의 선택 기준은 아주 간단하게 정리할 수 있다.

이 작업은 "값 치환"인가, "규칙 판단"인가?
- 값 → 값 치환이면 map
- 판단·조건·계산 로직이 필요하면 apply